#### Tandem model using detect and classify models from Yu, et al
Tandem inference implementation of the detection and classification model from the Yu, et al [paper](https://drive.google.com/file/d/1nYl4w41CAcj8XwTEdVwcD5lVheUFIHVy/view?usp=sharing)

In [ ]:
!pip install pickle5

In [ ]:
# import libraries
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, losses, optimizers, regularizers, callbacks

import os
import time
import numpy as np
import pandas as pd

import helpers as helper
from keras_model_s3_wrapper import *

import boto3
import pickle5 as pickle
s3 = boto3.resource('s3')
bucket_name = 'wafer-capstone'
my_bucket = s3.Bucket(bucket_name)

In [ ]:
tf.__version__

In [ ]:
tf.config.list_physical_devices(device_type=None)

In [ ]:
# specify variables
path = 'processed_data/WM-clean-paper'
result_path = 'results'

filename = 'WM-clean-paper'
map_column = 'waferMap224'

model_id = 'yutandem'
data_id = 'paper'
note = '' # -optional

detect_model = 'yudetect-paper'
classify_model = 'yuclassify-paper'

In [ ]:
# load test set(s) to be used
# directly from S3 (using boto3 resource)
start = time.time()

test_key = f'{path}/{filename}-test.pkl'
test = pickle.loads(my_bucket.Object(test_key).get()['Body'].read())

print("Wall time: {:.2f} seconds".format(time.time() - start))
print(f"Test: {len(test)}")

#### Quick EDA

In [ ]:
# baseline accuracy of test set
nones = len(test[test.detectLabels == 0])
total = len(test)
print(f"Baseline accuracy: {nones/total*100:.2f}%")

In [ ]:
# test failure type distribution
test_defects = test[test.detectLabels == 1]
helper.defect_distribution(test_defects, note='Test Set')

#### Detect Model

##### Detect data set-up

In [ ]:
# prepare inputs
start = time.time()

x_det = np.stack(test[map_column])
x_det = tf.expand_dims(x_det, axis=3, name=None)

print("Wall time: {:.2f} seconds".format(time.time() - start))
# sanity check
# expected: TensorShape([#rows, xdim, ydim, 1])
x_det.shape

In [ ]:
# prepare labels for evaluating results
start = time.time()

y_det = np.asarray(test['detectLabels']).astype(np.uint8)

print("Wall time: {:.2f} seconds".format(time.time() - start))

##### Load and run detect model

In [ ]:
# load saved detect model from S3
start = time.time()

detect = s3_get_keras_model(detect_model)
detect.summary()

print("Wall time: {:.2f} seconds".format(time.time() - start))

In [ ]:
# generate predictions
start = time.time()

detect_pred = detect.predict(x_det)
det_labels = np.argmax(detect_pred, axis=1).astype(np.uint8)

print("Wall time: {:.2f} seconds".format(time.time() - start))

In [ ]:
# compute model results on test set
start = time.time()

detect_results = model.evaluate(x_det, y_det)
detect_accuracy = detect_results[1] * 100

print("Wall time: {:.2f} seconds".format(time.time() - start))
print(f'Detect Model Loss: {detect_results[0]:.3f}')
print(f'Detect Model Accuracy: {detect_accuracy:.2f}')

#### Classify Model

##### Classify data set-up

In [ ]:
# keep only subset of test data
# predicted by detect model as having defects
defect_indices = [i for i in len(det_labels) if det_labels[i] == 1]
defect_ids = [test.ID[i] for i in defect_indices]
defect_df = test.loc[defect_indices].reset_index(drop=True)

# sanity check:
print(f'{len(defect_indices)}, {len(defect_ids)}, {defect_df.shape}')

In [ ]:
# prepare inputs
start = time.time()

x_cls = np.stack(defect_df[map_column])
x_cls = tf.expand_dims(x_cls, axis=3, name=None)

print("Wall time: {:.2f} seconds".format(time.time() - start))
# sanity check
# expected: TensorShape([#rows, xdim, ydim, 1])
x_det.shape

In [ ]:
# prepare labels for evaluating results
start = time.time()

y_cls = np.asarray(defect_df['classifyLabels']).astype(np.uint8)

print("Wall time: {:.2f} seconds".format(time.time() - start))

##### Load and run classify model

In [ ]:
# load saved classify model from S3
start = time.time()

classify = s3_get_keras_model(classify_model)
classify.summary()

print("Wall time: {:.2f} seconds".format(time.time() - start))

In [ ]:
# generate predictions
start = time.time()

classify_pred = detect.predict(x_cls)
cls_labels = np.argmax(classify_pred, axis=1).astype(np.uint8)

print("Wall time: {:.2f} seconds".format(time.time() - start))

In [ ]:
# generate full prediction
def tandem_prediction(x):
    if x in set(defect_ids):
        i = defect_ids.index(x)
        return cls_labels[i]
    else:
        return 8

test['tandemLabels'] = test.ID.apply(lambda x: tandem_prediction(x))
tandem_pred = test['tandemLabels'].tolist()
print(len(tandem_pred))

In [ ]:
# save predictions to local instance
predictions = [defect_ids, detect_pred, classify_pred, tandem_pred]
with open(f'{result_path}/{model_id}-{data_id}{note}.pkl', "wb") as f:
    pickle.dump(predictions, f)

#### Tandem model results

In [ ]:
# plot confusion matrix
y_test = test['classifyLabels'].tolist()
helper.plot_confusion_matrix(y_test, tandem_pred, mode='all', normalize=True)

In [ ]:
# plot confusion matrix counts
helper.plot_confusion_matrix(y_test, tandem_pred, mode='all', normalize=False)